In [3]:
import numpy as np
import torch
from neural_networks.models.LIM_class import *
from neural_networks.utilities import *
import cftime

%load_ext autoreload
%autoreload 2

# Create the DataLoader for first principal component
data = torch.load("./neural_networks/data/data_piControl.pt")
print("Data shape : {}".format(data.shape))
print(min_max_values_per_slice(data))
data = normalize_data(data)
print(min_max_values_per_slice(data))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Data shape : torch.Size([30, 14400])
{'slice_0': {'min': -145.2110595703125, 'max': 162.90086364746094}, 'slice_1': {'min': -65.64151763916016, 'max': 69.48644256591797}, 'slice_2': {'min': -62.16381072998047, 'max': 79.00810241699219}, 'slice_3': {'min': -41.96421432495117, 'max': 61.22380065917969}, 'slice_4': {'min': -36.37443542480469, 'max': 41.21829605102539}, 'slice_5': {'min': -41.3738899230957, 'max': 50.38222885131836}, 'slice_6': {'min': -36.039527893066406, 'max': 37.63322067260742}, 'slice_7': {'min': -32.235721588134766, 'max': 37.108612060546875}, 'slice_8': {'min': -32.59757614135742, 'max': 36.5982780456543}, 'slice_9': {'min': -30.33255958557129, 'max': 37.053401947021484}, 'slice_10': {'min': -31.394615173339844, 'max': 31.727630615234375}, 'slice_11': {'min': -28.008262634277344, 'max': 29.675642013549805}, 'slice_12': {'min': -25.62923812866211, 'max': 26.046384811401367}, 'slic

In [ ]:
timesteps = 60000
time = True
# original fit
tau = 1
model_org = LIM(tau)
model_org.fit(data.detach().cpu().numpy())
#131999 = 11000 years
lim_integration, times_ = model_org.noise_integration(data[:, -1], timesteps=timesteps, num_comp=30)
lim_integration = lim_integration.T

lim_data = torch.zeros(lim_integration.shape[0]+1, lim_integration.shape[1])

if time is True:
    count = 0
    for t in range(int(timesteps / 12)):
        for m in range(12):
            month = np.array(m+1)
            month = np.expand_dims(month, axis=0)
            lim = np.append(lim_integration[:, count + m], month, axis=0)
            lim_data[:, count+m] = torch.from_numpy(lim)
        count += 12
    lim_integration = lim_data

if time is False:
    lim_integration = torch.from_numpy(lim_integration)
torch.save(lim_integration, "./neural_networks/synthetic_data/lim_integration_TIME_120k2[-1]p.pt")
print("Saved data")

In [ ]:
timesteps = 20000
time = True
# original fit
tau = 1
model1 = LIM(tau)
model1.fit(data[:, :12000].detach().cpu().numpy())

model2 = LIM(tau)
model2.fit(data[:, :11000].detach().cpu().numpy())

model3 = LIM(tau)
model3.fit(data[:, :10000].detach().cpu().numpy())

model4 = LIM(tau)
model4.fit(data[:, :9000].detach().cpu().numpy())
#131999 = 11000 years
lim_integration1, times_ = model1.noise_integration(data[:, 12000], timesteps=timesteps, num_comp=30)
lim_integration1 = torch.from_numpy(lim_integration1.T)

lim_integration2, times_ = model2.noise_integration(data[:, 11000], timesteps=timesteps, num_comp=30)
lim_integration2 = torch.from_numpy(lim_integration2.T)

lim_integration3, times_ = model3.noise_integration(data[:, 10000], timesteps=timesteps, num_comp=30)
lim_integration3 = torch.from_numpy(lim_integration3.T)

lim_integration4, times_ = model4.noise_integration(data[:, 9000], timesteps=timesteps, num_comp=30)
lim_integration4 = torch.from_numpy(lim_integration4.T)

lim_integration = torch.cat((lim_integration1, lim_integration2, lim_integration3, lim_integration4), dim=1)
print(lim_integration.shape)

print(min_max_values_per_slice(lim_integration1))
print(min_max_values_per_slice(lim_integration2))
print(min_max_values_per_slice(lim_integration3))
print(min_max_values_per_slice(lim_integration4))
print(min_max_values_per_slice(lim_integration))
torch.save(lim_integration, "./neural_networks/synthetic_data/lim_integration_multipleLim.pt")


In [ ]:
eofs = np.arange(0, 30)

date_end = (timesteps / 12) + 12
# Set the start and end dates
start_date = cftime.DatetimeNoLeap(1, 1, 15, 12, 0, 0, 0, has_year_zero=True)
end_date = cftime.DatetimeNoLeap(date_end, 10, 15, 12, 0, 0, 0, has_year_zero=True)

# Create a range of monthly timestamps
time = xr.cftime_range(start=start_date, end=end_date, freq='M')

print("Lim interation : {} {}".format(type(lim_integration), lim_integration.shape))
# Create a DataArray from the numpy array with coordinates
data_xr = xr.DataArray(lim_integration, coords=[np.arange(30), time], dims=['eof', 'time'])

# Print the created xarray
print(type(data_xr))

#data_xr.to_netcdf('./neural_networks/synthetic_data/lim_integration_xarray_20k[-1]p.nc')
print("Save data")